In [0]:
# Dependencies to load moduels from this repo
import importlib.util
import sys

# Load cv module directly from file path
cv_path = "/Workspace/Shared/Team 4_2/flight-departure-delay-predictive-modeling/notebooks/Cross Validator/cv.py"
spec = importlib.util.spec_from_file_location("cv", cv_path)
cv = importlib.util.module_from_spec(spec)
spec.loader.exec_module(cv)

# Dependencies for graph features
from graphframes import *
from pyspark.sql import functions as F
from pyspark.sql.functions import col

# Other Dependencies
import time

# Path for persistent storage
FOLDER_PATH = "dbfs:/mnt/mids-w261/student-groups/Group_4_2/experiments"

## Use Cross Validator Module to Generate Folds

In [0]:
data_loader = cv.FlightDelayDataLoader()
data_loader.load()

In [0]:
folds = data_loader.get_version("3M")

In [0]:
# Get training data from first fold to build graph
train_df, val_df = folds[0]

## Generate Graph from One Training Fold

### Graph Construction
Origin Airport --Flight--> Destination Airport

* **Nodes**: Airport Codes
* **Edges**: Flights
  * **Direction**: Origin to Destination
  * **Weight**: Number of Flights

In [0]:
# Build graph: nodes are airports, edges are flights (origin -> dest)
# Edge weights = number of flights between airports

# Create edges: (origin, dest) with count as weight
edges = (
    train_df
    .select("origin", "dest")
    .filter(col("origin").isNotNull() & col("dest").isNotNull())
    .groupBy("origin", "dest")
    .count()
    .withColumnRenamed("origin", "src")
    .withColumnRenamed("dest", "dst")
    .withColumnRenamed("count", "weight")
)

display(edges.limit(10))

In [0]:
# Checkpoint to run notebook more quickly in the future
edges_path = f"{FOLDER_PATH}/graph_edges.parquet"
edges.write.mode("overwrite").parquet(edges_path)

In [0]:
# If re-running this notebook, start here as edges are checkpointed
edges = spark.read.parquet(edges_path)
edges.count()  # Materialize
display(edges.limit(10))

In [0]:
# Create vertices: all unique airports (both origin and destination)
src_airports = edges.select(col("src").alias("id")).distinct()
dst_airports = edges.select(col("dst").alias("id")).distinct()
vertices = src_airports.union(dst_airports).distinct()

print(f"Number of airports (vertices): {vertices.count()}")
print(f"Number of routes (edges): {edges.count()}")

### Generate Graphframes
**NOTE**: GraphFrames PageRank does NOT use edge weights (HW5 Q5.f). It treats all edges equally (weight = 1), ignoring the "weight" column. For weighted PageRank, we'd need to use the RDD approach from HW5 Q4

**Correction**: GraphFrames automatically detects a column named weight in the edges DataFrame. Pagerank uses weights by default


In [0]:
# Create GraphFrame
start = time.time()
g = GraphFrame(vertices, edges)
print(f"Ran in {time.time() - start:.2f} seconds")

In [0]:
# Verify edges DataFrame has "weight" column (GraphFrames automatically detects this)
print("Edges DataFrame schema:")
edges.printSchema()
print("\nSample edges with weights:")
display(edges.limit(5))

### "Weighted" GraphFrame Workaround

In [0]:
# Create weighted graph using duplication workaround
# Duplicate each edge based on its weight (e.g., weight=3 means 3 copies of the edge)
# This simulates weighted PageRank since GraphFrames treats all edges equally
start = time.time()

# Get edges with weight column
edges_with_weights = g.edges

# Create sequence array [0, 1, 2, ..., weight-1] for each edge, then explode to duplicate
edges_weighted = (
    edges_with_weights
    .withColumn("seq", F.sequence(F.lit(0), F.col("weight").cast("int") - 1))
    .select("src", "dst", F.explode("seq").alias("_"))
    .select("src", "dst")
)

# Create weighted GraphFrame (same vertices, duplicated edges)
g_weighted = GraphFrame(vertices, edges_weighted)

print(f"Created weighted GraphFrame (duplication workaround) in {time.time() - start:.2f} seconds")
print(f"Original edges: {edges.count()}")
print(f"Weighted edges (after duplication): {edges_weighted.count()}")

## Graph Analysis

In [0]:
# Set checkpoint directory (required for GraphFrames algorithms like connectedComponents)
sc = spark.sparkContext
sc.setCheckpointDir("dbfs:/tmp/graphframes_checkpoint")

In [0]:
# Connectivity Analysis (referencing HW5 concepts)
# Check for connected components (islands) - weakly connected for directed graph
connected_components = g.connectedComponents()

# Count number of distinct components
num_components = connected_components.select("component").distinct().count()
print(f"Number of connected components (islands): {num_components}")

# Show component sizes
component_sizes = (
    connected_components
    .groupBy("component")
    .count()
    .orderBy(F.desc("count"))
)

print("\nTop 10 largest components:")
display(component_sizes.limit(10))

# Check if graph is strongly connected (all nodes reachable from all nodes)
# For directed graphs, we check strongly connected components
strongly_connected = g.stronglyConnectedComponents(maxIter=10)
num_strong_components = strongly_connected.select("component").distinct().count()
print(f"\nNumber of strongly connected components: {num_strong_components}")

In [0]:
# Check for dangling nodes (nodes with no outlinks) - key issue from HW5 Q2.b
# Dangling nodes are nodes that receive links but don't link to anything else
out_degree = g.outDegrees
in_degree = g.inDegrees

# Find nodes with out-degree = 0 (dangling nodes)
dangling_nodes = (
    vertices
    .join(out_degree, "id", "left_outer")
    .filter(col("outDegree").isNull() | (col("outDegree") == 0))
)

num_dangling = dangling_nodes.count()
print(f"Number of dangling nodes (no outlinks): {num_dangling}")

if num_dangling > 0:
    print("\nSample dangling nodes:")
    display(dangling_nodes.limit(10))


In [0]:
# Check for nodes with no inlinks (sources)
source_nodes = (
    vertices
    .join(in_degree, "id", "left_outer")
    .filter(col("inDegree").isNull() | (col("inDegree") == 0))
)

num_sources = source_nodes.count()
print(f"Number of source nodes (no inlinks): {num_sources}")

# Summary statistics
total_nodes = vertices.count()
print(f"\n=== Graph Connectivity Summary ===")
print(f"Total nodes: {total_nodes}")
print(f"Weakly connected components: {num_components}")
print(f"Strongly connected components: {num_strong_components}")
print(f"Dangling nodes (no outlinks): {num_dangling}")
print(f"Source nodes (no inlinks): {num_sources}")

## Run PageRank on Connectivity Graph

In [0]:
start = time.time()

# Run PageRank **without** edge weights
pagerank_results_unweighted = g.pageRank(resetProbability=0.15, maxIter=10)

# Display top airports by PageRank
top_airports_unweighted = pagerank_results_unweighted.vertices.orderBy(F.desc("pagerank")).limit(20)
display(top_airports_unweighted)

print(f"Ran in {time.time() - start:.2f} seconds")

## Flight Sequence Graph (For Visualization/Analysis)

### ⚠️ Important: GraphFrames vs Window Functions

**Key Insight**: For Flight Lineage feature engineering, we DON'T need GraphFrames!

The features we need (previous flight info, cumulative delays, conditional expected values) are all:
- **Window functions** (LAG, SUM, AVG over partitions)
- **Joins** (with pre-computed conditional expected value tables)
- **Conditional logic** (IF-THEN calculations)

**Graph algorithms** (PageRank, shortest paths, etc.) are NOT needed.

**See**: `FLIGHT_LINEAGE_DESIGN.md` for detailed analysis.

### When to Use This Graph

This graph is useful for:
- **Visualization**: Understanding flight sequences
- **Analysis**: Jump patterns, sequence statistics
- **NOT for feature engineering**: Use window functions instead

### Sequential Graph Structure (No Self-Loops)

To avoid self-loops and create a strictly sequential graph:

**Nodes**: `{tail_num}_{FL_DATE}_{seq_num}` or `{tail_num}_{FL_DATE}_{crs_dep_time}`
- Ensures uniqueness
- Preserves temporal ordering
- No self-loops

**Edges**: `(node_i, node_i+1)` where nodes are consecutive flights
- `src`: `{tail_num}_{date}_{seq_num}`
- `dst`: `{tail_num}_{date}_{seq_num+1}`
- Edge attributes: `dest_airport`, `air_time`, `is_jump`, etc.

**Example**: A→B→A→C [Break] D→E becomes:
- Node1: `N12345_2023-01-15_1` (A→B)
- Node2: `N12345_2023-01-15_2` (B→A)
- Node3: `N12345_2023-01-15_3` (A→C)
- [Break/jump]
- Node4: `N12345_2023-01-15_4` (D→E)


In [ ]:
# Build Sequential Flight Sequence Graph (for visualization/analysis)
# Nodes: {tail_num}_{FL_DATE}_{seq_num} to avoid self-loops
# This creates a strictly sequential graph: A→B→A→C [Break] D→E

print("Building Sequential Flight Sequence Graph...")
print("Nodes: {tail_num}_{FL_DATE}_{seq_num} (unique flight identifiers)")
print("Edges: Consecutive flights in sequence (no self-loops)\n")
print("NOTE: For feature engineering, use window functions instead of this graph.\n")

# Check if tail_num column exists
tail_cols = [c for c in train_df.columns if 'tail' in c.lower()]
print(f"Tail number columns found: {tail_cols}")

if not tail_cols:
    print("WARNING: No tail_num column found. Skipping flight sequence graph.")
else:
    tail_col = tail_cols[0]  # Use first matching column
    print(f"Using column: {tail_col}\n")
    
    # Prepare data: need tail_num, FL_DATE, origin, dest, scheduled times, delays
    required_cols = [tail_col, 'FL_DATE', 'origin', 'dest', 'crs_dep_time']
    available_cols = [c for c in required_cols if c in train_df.columns]
    
    if len(available_cols) < len(required_cols):
        missing = set(required_cols) - set(available_cols)
        print(f"WARNING: Missing columns: {missing}")
        print("Skipping flight sequence graph.")
    else:
        # Select and filter flight data
        flight_seq_data = (
            train_df
            .select(
                tail_col, 'FL_DATE', 'origin', 'dest', 
                'crs_dep_time', 'air_time', 'DEP_DELAY', 'ARR_DELAY'
            )
            .filter(
                col(tail_col).isNotNull() & 
                col('FL_DATE').isNotNull() &
                col('origin').isNotNull() & 
                col('dest').isNotNull() &
                col('crs_dep_time').isNotNull()
            )
        )
        
        print(f"Total flights with valid tail_num: {flight_seq_data.count():,}")
        
        # Order flights by tail_num, date, and scheduled departure time
        # This ensures we track sequences correctly (A→B→A→C, not A→B & A→C)
        from pyspark.sql.window import Window
        
        window_spec = Window.partitionBy(tail_col, 'FL_DATE').orderBy('crs_dep_time')
        
        # Add sequence number and previous flight information
        flight_seq_ordered = (
            flight_seq_data
            .withColumn('seq_num', F.row_number().over(window_spec))
            .withColumn('prev_dest', F.lag('dest', 1).over(window_spec))
            .withColumn('prev_air_time', F.lag('air_time', 1).over(window_spec))
            .withColumn('prev_arr_delay', F.lag('ARR_DELAY', 1).over(window_spec))
        )
        
        # Detect jumps: when prev_dest != current_origin (or prev_dest is null for first flight)
        flight_seq_with_jumps = flight_seq_ordered.withColumn(
            'is_jump',
            when(col('seq_num') == 1, F.lit(False))  # First flight is not a jump
            .when(col('prev_dest').isNull(), F.lit(True))  # Missing previous flight = jump
            .otherwise(col('prev_dest') != col('origin'))  # prev_dest != origin = jump
        )
        
        # Create unique node IDs: {tail_num}_{FL_DATE}_{seq_num}
        # This ensures no self-loops and preserves temporal ordering
        flight_seq_with_nodes = flight_seq_with_jumps.withColumn(
            'node_id',
            F.concat(
                col(tail_col), F.lit('_'),
                col('FL_DATE'), F.lit('_'),
                col('seq_num')
            )
        )
        
        # Create edges: (node_i, node_i+1) for consecutive flights
        # Only create edges between consecutive flights (seq_num difference = 1)
        flight_seq_edges = (
            flight_seq_with_nodes
            .withColumn('next_node_id', F.lead('node_id', 1).over(window_spec))
            .filter(col('next_node_id').isNotNull())  # Skip last flight in sequence
            .select(
                col('node_id').alias('src'),  # Current flight node
                col('next_node_id').alias('dst'),  # Next flight node
                # Edge attributes:
                col('origin').alias('origin_airport'),
                col('dest').alias('dest_airport'),
                col('FL_DATE').alias('flight_date'),
                col('crs_dep_time').alias('scheduled_dep_time'),
                col('air_time').alias('air_time'),
                col('is_jump').alias('is_jump'),
                col('DEP_DELAY').alias('dep_delay'),
                col('ARR_DELAY').alias('arr_delay'),
                col('seq_num').alias('seq_num'),
                col('prev_dest').alias('prev_dest_airport'),
                col('prev_air_time').alias('prev_air_time'),
                col('prev_arr_delay').alias('prev_arr_delay')
            )
        )
        
        # Create vertices: all unique node IDs
        flight_seq_vertices = (
            flight_seq_with_nodes
            .select(
                col('node_id').alias('id'),
                col(tail_col).alias('tail_num'),
                col('FL_DATE').alias('flight_date'),
                col('seq_num').alias('seq_num'),
                col('origin').alias('origin_airport'),
                col('dest').alias('dest_airport'),
                col('crs_dep_time').alias('scheduled_dep_time')
            )
            .distinct()
        )
        
        print(f"\nFlight sequence edges created: {flight_seq_edges.count():,}")
        print("\nSample flight sequence edges (showing sequential structure, no self-loops):")
        display(flight_seq_edges.select('src', 'dst', 'origin_airport', 'dest_airport', 
                                        'seq_num', 'is_jump', 'prev_dest_airport').limit(20))
        
        print(f"\nFlight sequence nodes (unique flight identifiers): {flight_seq_vertices.count():,}")
        print("\nSample nodes:")
        display(flight_seq_vertices.limit(10))
        
        # Analyze jump patterns
        jump_stats = (
            flight_seq_edges
            .groupBy('is_jump')
            .agg(
                F.count('*').alias('count'),
                F.avg('dep_delay').alias('avg_dep_delay'),
                F.avg('arr_delay').alias('avg_arr_delay')
            )
        )
        print("\nJump Statistics:")
        display(jump_stats)
        
        # Save for later use
        flight_seq_edges_path = f"{FOLDER_PATH}/flight_sequence_edges.parquet"
        flight_seq_vertices_path = f"{FOLDER_PATH}/flight_sequence_vertices.parquet"
        
        flight_seq_edges.write.mode("overwrite").parquet(flight_seq_edges_path)
        flight_seq_vertices.write.mode("overwrite").parquet(flight_seq_vertices_path)
        
        print(f"\nSaved flight sequence graph components:")
        print(f"  Edges: {flight_seq_edges_path}")
        print(f"  Vertices: {flight_seq_vertices_path}")


In [ ]:
# Load flight sequence graph (if re-running)
flight_seq_edges_path = f"{FOLDER_PATH}/flight_sequence_edges.parquet"
flight_seq_vertices_path = f"{FOLDER_PATH}/flight_sequence_vertices.parquet"

try:
    flight_seq_edges = spark.read.parquet(flight_seq_edges_path)
    flight_seq_vertices = spark.read.parquet(flight_seq_vertices_path)
    
    print(f"Loaded flight sequence graph:")
    print(f"  Edges: {flight_seq_edges.count():,}")
    print(f"  Vertices: {flight_seq_vertices.count():,}")
    
    print("\nSample edges (showing sequence and jump detection):")
    display(flight_seq_edges.select('src', 'dst', 'origin_airport', 'dest_airport', 
                                    'seq_num', 'is_jump', 'prev_dest_airport').limit(10))
    
    # Show jump statistics
    jump_stats = (
        flight_seq_edges
        .groupBy('is_jump')
        .agg(F.count('*').alias('count'))
    )
    print("\nJump Statistics:")
    display(jump_stats)
except Exception as e:
    print(f"Could not load flight sequence graph: {e}")
    print("Run the previous cell to build it first.")


In [ ]:
### GraphFrame Creation and Limitations

**Note**: This graph uses self-loops (edges from tail_num to the same tail_num). GraphFrames may have limitations with self-loops for certain algorithms like PageRank. We'll test and document any issues.
    start = time.time()
    
    g_flight_seq = GraphFrame(flight_seq_vertices, flight_seq_edges_agg)
    
    print(f"Created GraphFrame in {time.time() - start:.2f} seconds")
    print(f"  Vertices: {g_flight_seq.vertices.count():,}")
    print(f"  Edges: {g_flight_seq.edges.count():,}")
    
    # Compute basic graph statistics
    print("\n=== Flight Sequence Graph Statistics ===")
    
    # Connected components
    connected_components_fs = g_flight_seq.connectedComponents()
    num_components_fs = connected_components_fs.select("component").distinct().count()
    print(f"Connected components: {num_components_fs}")
    
    # In/out degrees
    in_degree_fs = g_flight_seq.inDegrees
    out_degree_fs = g_flight_seq.outDegrees
    
    print(f"Nodes with in-degree > 0: {in_degree_fs.count():,}")
    print(f"Nodes with out-degree > 0: {out_degree_fs.count():,}")
    
    # Sample nodes
    print("\nSample nodes (first 10):")
    display(g_flight_seq.vertices.limit(10))
else:
    print("Flight sequence graph not available. Run previous cells to build it.")


In [ ]:
# Create GraphFrame for flight sequence graph
# NOTE: This graph has self-loops (src == dst), which may limit some GraphFrames algorithms

if 'flight_seq_edges' in locals() and 'flight_seq_vertices' in locals():
    print("Creating GraphFrame for flight sequence graph...")
    print("WARNING: This graph contains self-loops (same tail_num as src and dst)")
    print("Some GraphFrames algorithms (e.g., PageRank) may not work correctly with self-loops.\n")
    
    start = time.time()
    
    try:
        g_flight_seq = GraphFrame(flight_seq_vertices, flight_seq_edges)
        
        print(f"Created GraphFrame in {time.time() - start:.2f} seconds")
        print(f"  Vertices: {g_flight_seq.vertices.count():,}")
        print(f"  Edges: {g_flight_seq.edges.count():,}")
        
        # Check for self-loops
        self_loops = (
            g_flight_seq.edges
            .filter(col('src') == col('dst'))
            .count()
        )
        print(f"  Self-loops: {self_loops:,} ({100*self_loops/g_flight_seq.edges.count():.1f}%)")
        
        # Compute basic graph statistics
        print("\n=== Flight Sequence Graph Statistics ===")
        
        # In/out degrees
        in_degree_fs = g_flight_seq.inDegrees
        out_degree_fs = g_flight_seq.outDegrees
        
        print(f"Nodes with in-degree > 0: {in_degree_fs.count():,}")
        print(f"Nodes with out-degree > 0: {out_degree_fs.count():,}")
        
        # Sample nodes
        print("\nSample nodes (first 10):")
        display(g_flight_seq.vertices.limit(10))
        
        # Sample edges with attributes
        print("\nSample edges with attributes:")
        display(g_flight_seq.edges.select('src', 'dst', 'origin_airport', 'dest_airport', 
                                          'seq_num', 'is_jump').limit(10))
        
        # Test PageRank (may fail with self-loops)
        print("\n=== Testing PageRank (may fail with self-loops) ===")
        try:
            pagerank_fs = g_flight_seq.pageRank(resetProbability=0.15, maxIter=10)
            print("✓ PageRank completed successfully")
            print("\nTop 10 tail_nums by PageRank:")
            display(pagerank_fs.vertices.orderBy(F.desc('pagerank')).limit(10))
        except Exception as e:
            print(f"✗ PageRank failed (likely due to self-loops): {e}")
            print("  This is expected - we may need a different graph structure for PageRank")
        
    except Exception as e:
        print(f"Error creating GraphFrame: {e}")
        print("This may be due to self-loops or other graph structure issues.")
else:
    print("Flight sequence graph not available. Run previous cells to build it.")


## Feature Engineering Approach (Window Functions)

**For actual feature engineering, use window functions instead of the graph:**

This section demonstrates how to compute Flight Lineage features using window functions, which is the recommended approach.


In [ ]:
# Example: Flight Lineage Feature Engineering using Window Functions
# This is the RECOMMENDED approach for feature engineering

if 'train_df' in locals():
    print("Computing Flight Lineage features using window functions...")
    
    # Find tail_num column
    tail_cols = [c for c in train_df.columns if 'tail' in c.lower()]
    if tail_cols:
        tail_col = tail_cols[0]
        print(f"Using tail_num column: {tail_col}\n")
        
        # Prepare data
        lineage_features = (
            train_df
            .select(
                tail_col, 'FL_DATE', 'origin', 'dest', 'crs_dep_time',
                'DEP_DELAY', 'ARR_DELAY', 'air_time', 'op_carrier'
            )
            .filter(
                col(tail_col).isNotNull() & 
                col('FL_DATE').isNotNull() &
                col('origin').isNotNull() & 
                col('dest').isNotNull() &
                col('crs_dep_time').isNotNull()
            )
        )
        
        # Window specification: partition by tail_num and date, order by scheduled departure time
        from pyspark.sql.window import Window
        window_spec = Window.partitionBy(tail_col, 'FL_DATE').orderBy('crs_dep_time')
        
        # 1. Previous flight information (using LAG)
        lineage_features = lineage_features.withColumn(
            'prev_flight_dest', F.lag('dest', 1).over(window_spec)
        ).withColumn(
            'prev_flight_arr_delay', F.lag('ARR_DELAY', 1).over(window_spec)
        ).withColumn(
            'prev_flight_dep_delay', F.lag('DEP_DELAY', 1).over(window_spec)
        ).withColumn(
            'prev_flight_air_time', F.lag('air_time', 1).over(window_spec)
        )
        
        # 2. Sequence number
        lineage_features = lineage_features.withColumn(
            'seq_num', F.row_number().over(window_spec)
        )
        
        # 3. Jump detection
        lineage_features = lineage_features.withColumn(
            'is_jump',
            when(col('seq_num') == 1, F.lit(False))
            .when(col('prev_flight_dest').isNull(), F.lit(True))
            .otherwise(col('prev_flight_dest') != col('origin'))
        )
        
        # 4. Cumulative delay (sum of previous flights' delays)
        lineage_features = lineage_features.withColumn(
            'cumulative_delay_since_3am',
            F.sum('DEP_DELAY').over(
                window_spec.rowsBetween(Window.unboundedPreceding, -1)
            )
        )
        
        # 5. Number of previous flights
        lineage_features = lineage_features.withColumn(
            'num_previous_flights_today',
            F.count('*').over(
                window_spec.rowsBetween(Window.unboundedPreceding, -1)
            )
        )
        
        # 6. Average delay per previous flight
        lineage_features = lineage_features.withColumn(
            'avg_delay_per_previous_flight',
            F.avg('DEP_DELAY').over(
                window_spec.rowsBetween(Window.unboundedPreceding, -1)
            )
        )
        
        print("✓ Flight Lineage features computed using window functions")
        print(f"\nSample features:")
        display_cols = [tail_col, 'FL_DATE', 'origin', 'dest', 'seq_num', 'is_jump',
                       'prev_flight_dest', 'prev_flight_arr_delay', 
                       'cumulative_delay_since_3am', 'num_previous_flights_today']
        display(lineage_features.select(display_cols).limit(10))
        
        print("\n=== Summary Statistics ===")
        jump_stats = (
            lineage_features
            .groupBy('is_jump')
            .agg(
                F.count('*').alias('count'),
                F.avg('cumulative_delay_since_3am').alias('avg_cumulative_delay')
            )
        )
        display(jump_stats)
        
        print("\nNOTE: For conditional expected values (turn time, air time),")
        print("      join with pre-computed tables from Time-Series Features Experiment.")
        
        print("\n" + "="*60)
        print("KEY INSIGHT: Pulling Engineered Features from Previous Flight")
        print("="*60)
        print("""
To use ENGINEERED features from the previous flight (not just raw values):
1. First, compute the feature for the current flight using previous flight's RAW values
2. Then, use LAG to pull that computed feature for the NEXT flight

Example:
  Flight 2: Compute expected_arrival_time using prev_actual_dep_time
  Flight 3: Use LAG(expected_arrival_time) to get prev_expected_arrival_time
  Flight 3: Compute expected_departure_time using prev_expected_arrival_time

This requires multiple passes with withColumn, but Spark handles this correctly.
        """)
    else:
        print("tail_num column not found. Skipping feature engineering example.")
else:
    print("train_df not available. Load data first.")


## Advanced: Pulling Engineered Features from Previous Flight

This example shows how to compute features that depend on **engineered features** from the previous flight, not just raw values.


In [ ]:
# Example: Computing features that depend on previous flight's ENGINEERED features
# This demonstrates the multi-pass approach

if 'train_df' in locals():
    print("Example: Computing features with dependencies on previous flight's engineered features...\n")
    
    tail_cols = [c for c in train_df.columns if 'tail' in c.lower()]
    if tail_cols:
        tail_col = tail_cols[0]
        
        # Prepare data
        df = (
            train_df
            .select(
                tail_col, 'FL_DATE', 'origin', 'dest', 'crs_dep_time',
                'dep_time', 'arr_time', 'DEP_DELAY', 'ARR_DELAY', 
                'air_time', 'op_carrier'
            )
            .filter(
                col(tail_col).isNotNull() & 
                col('FL_DATE').isNotNull() &
                col('origin').isNotNull() & 
                col('dest').isNotNull() &
                col('crs_dep_time').isNotNull()
            )
        )
        
        window_spec = Window.partitionBy(tail_col, 'FL_DATE').orderBy('crs_dep_time')
        
        # ============================================================
        # PASS 1: Pull RAW values from previous flight
        # ============================================================
        print("Pass 1: Pulling raw values from previous flight...")
        df = df.withColumn('seq_num', F.row_number().over(window_spec))
        df = df.withColumn('prev_actual_dep_time', F.lag('dep_time', 1).over(window_spec))
        df = df.withColumn('prev_air_time', F.lag('air_time', 1).over(window_spec))
        df = df.withColumn('prev_dest', F.lag('dest', 1).over(window_spec))
        
        # ============================================================
        # PASS 2: Compute ENGINEERED features for CURRENT flight
        #         using previous flight's RAW values
        # ============================================================
        print("Pass 2: Computing engineered features using previous flight's raw values...")
        
        # For this example, we'll use a simple expected air time (in real code, join with lookup table)
        # In production, you'd join with expected_air_time_route from Time-Series Features
        df = df.withColumn('expected_air_time_route', 
            when(col('air_time').isNotNull(), col('air_time'))  # Use actual if available
            .otherwise(F.lit(120))  # Placeholder - in production, join with lookup table
        )
        
        # Compute expected arrival time of PREVIOUS flight
        # This is computed for the CURRENT flight, but represents the previous flight's expected arrival
        df = df.withColumn('expected_arrival_time_prev_flight',
            when(
                (col('prev_actual_dep_time').isNotNull()) & 
                (col('expected_air_time_route').isNotNull()),
                col('prev_actual_dep_time') + col('expected_air_time_route')
            )
            .otherwise(None)
        )
        
        # ============================================================
        # PASS 3: Pull ENGINEERED features from previous flight
        #         for use in CURRENT flight's calculations
        # ============================================================
        print("Pass 3: Pulling engineered features from previous flight...")
        
        # Now pull the computed feature for use in current flight
        df = df.withColumn('prev_expected_arrival_time',
            F.lag('expected_arrival_time_prev_flight', 1).over(window_spec)
        )
        
        # ============================================================
        # PASS 4: Use previous flight's ENGINEERED features to compute
        #         current flight's features
        # ============================================================
        print("Pass 4: Computing current flight features using previous flight's engineered features...")
        
        # For this example, use a simple expected turn time (in production, join with lookup table)
        df = df.withColumn('expected_turn_time_carrier_airport', F.lit(45))  # Placeholder
        
        # Compute expected departure time for CURRENT flight
        # This uses the previous flight's EXPECTED ARRIVAL TIME (an engineered feature)
        df = df.withColumn('expected_departure_time_current',
            when(
                (col('prev_expected_arrival_time').isNotNull()) &
                (col('expected_turn_time_carrier_airport').isNotNull()),
                col('prev_expected_arrival_time') + col('expected_turn_time_carrier_airport')
            )
            .otherwise(None)
        )
        
        # Compute time buffer and impossible on-time flag
        df = df.withColumn('time_buffer',
            when(
                (col('expected_departure_time_current').isNotNull()) &
                (col('crs_dep_time').isNotNull()),
                col('crs_dep_time') - col('expected_departure_time_current')
            )
            .otherwise(None)
        )
        
        df = df.withColumn('impossible_on_time_flag',
            when(col('time_buffer').isNotNull(),
                 when(col('time_buffer') < 0, 1).otherwise(0))
            .otherwise(None)
        )
        
        print("\n✓ Multi-pass feature engineering complete!")
        print("\nSample results showing feature dependencies:")
        display_cols = [
            tail_col, 'seq_num', 'origin', 'dest',
            'prev_actual_dep_time',  # Raw value from prev flight
            'expected_arrival_time_prev_flight',  # Engineered for prev flight
            'prev_expected_arrival_time',  # Pulled engineered feature
            'expected_departure_time_current',  # Uses prev engineered feature
            'time_buffer', 'impossible_on_time_flag'
        ]
        display(df.select(display_cols).limit(10))
        
        print("\n" + "="*60)
        print("Key Points:")
        print("="*60)
        print("1. LAG can pull ANY column, including ones you just computed")
        print("2. Order matters: compute features in dependency order")
        print("3. Each withColumn can see results of previous withColumn calls")
        print("4. In production, join with lookup tables for expected values")
        print("5. Handle NULLs carefully (first flight, jumps, missing data)")
    else:
        print("tail_num column not found.")
else:
    print("train_df not available.")


In [0]:
# Run Weighted PageRank using the "weight" column (number of flights)
pagerank_results_weighted = g_weighted.pageRank(resetProbability=0.15, maxIter=10)

# Display top airports by PageRank
top_airports_weighted = pagerank_results_weighted.vertices.orderBy(F.desc("pagerank")).limit(20)
display(top_airports_weighted)